## MLflow's Model Registry

In [4]:
from mlflow.tracking import MlflowClient


MLFLOW_TRACKING_URI = "sqlite:///../mlflow.db"

### Interacting with the MLflow tracking server

The `MlflowClient` object allows us to interact with...
- an MLflow Tracking Server that creates and manages experiments and runs.
- an MLflow Registry Server that creates and manages registered models and model versions. 

To instantiate it we need to pass a tracking URI and/or a registry URI

In [5]:
client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)

client.search_experiments()

INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.


[<Experiment: artifact_location='/workspaces/mlops-zoomcamp/02-experiment-tracking/mlruns/2', creation_time=1718877908629, experiment_id='2', last_update_time=1718877908629, lifecycle_stage='active', name='my-cool-experiment', tags={}>,
 <Experiment: artifact_location='/workspaces/mlops-zoomcamp/02-experiment-tracking/mlruns/1', creation_time=1718809930065, experiment_id='1', last_update_time=1718809930065, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>,
 <Experiment: artifact_location='mlflow-artifacts:/0', creation_time=1718808575988, experiment_id='0', last_update_time=1718808575988, lifecycle_stage='active', name='Default', tags={}>]

In [7]:
client.create_experiment(name="my-cool-experiment2")

'3'

Let's check the latest versions for the experiment with id `1`...

In [10]:
from mlflow.entities import ViewType

runs = client.search_runs(
    experiment_ids=['1'],
    filter_string="metrics.rmse < 7",
    run_view_type=ViewType.ACTIVE_ONLY,
    max_results=5,
    order_by=["metrics.rmse ASC"]
)

In [11]:
len(runs)

5

In [12]:
for run in runs:
    print(f"run id: {run.info.run_id}, rmse: {run.data.metrics['rmse']:.4f}")

run id: 18f755f5614c4fd2be976fa7bfb9a0e1, rmse: 5.1253
run id: a8f9de6fbe5544f28ba9053fe8be563e, rmse: 5.1262
run id: dddf954d8414489c9f37812b9fd019b6, rmse: 5.1323
run id: 288657453abd46fa8517f531acebc132, rmse: 5.1343
run id: 0d4fea5fc5de4971afff2f6ef5dc0a55, rmse: 5.1369


### Interacting with the Model Registry

In this section We will use the `MlflowClient` instance to:

1. Register a new version for the experiment `nyc-taxi-regressor`
2. Retrieve the latests versions of the model `nyc-taxi-regressor` and check that a new version `4` was created.
3. Transition the version `4` to "Staging" and adding annotations to it.

In [13]:
import mlflow

mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)

In [29]:
run_id = "f5520aca4b114fb199caa353f566ab35"
model_uri = f"runs:/{run_id}/model"
mlflow.register_model(model_uri=model_uri, name="nyc_taxi_model")

Registered model 'nyc_taxi_model' already exists. Creating a new version of this model...
Created version '8' of model 'nyc_taxi_model'.


<ModelVersion: aliases=[], creation_timestamp=1718882808109, current_stage='None', description=None, last_updated_timestamp=1718882808109, name='nyc_taxi_model', run_id='f5520aca4b114fb199caa353f566ab35', run_link=None, source='/workspaces/mlops-zoomcamp/02-experiment-tracking/mlruns/1/f5520aca4b114fb199caa353f566ab35/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=8>

In [15]:
model_name = "nyc_taxi_model"
#latest_versions = client.get_latest_versions(name=model_name, )

client.set_registered_model_alias(model_name, "Sigma", "1")

client.set_registered_model_alias(model_name, "Sigma", "2")



sigma_model = client.get_model_version_by_alias(model_name, "Sigma")

# Set registered model tag 
client.set_registered_model_tag(model_name, "task", "classification")
# Set model version tag
client.set_model_version_tag(model_name, "1", "validation_status", "approved")
# for version in latest_versions:
#     print(f"alias: {version.aliases}, tags: {version.tags}")

In [18]:
versions = client.get_registered_model(model_name)

In [19]:
versions

<RegisteredModel: aliases={'Sigma': 2}, creation_timestamp=1718877186169, description='', last_updated_timestamp=1718881468199, latest_versions=[<ModelVersion: aliases=[], creation_timestamp=1718881468199, current_stage='None', description=None, last_updated_timestamp=1718881468199, name='nyc_taxi_model', run_id='18f755f5614c4fd2be976fa7bfb9a0e1', run_link=None, source='/workspaces/mlops-zoomcamp/02-experiment-tracking/mlruns/1/18f755f5614c4fd2be976fa7bfb9a0e1/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=5>], name='nyc_taxi_model', tags={'task': 'classification'}>

In [20]:
model_version = "4"
new_stage = "Beta"
client.set_registered_model_alias(
    name=model_name,
    version=model_version,
    alias=new_stage)

In [21]:
from datetime import datetime

date = datetime.today().date()
client.update_model_version(
    name=model_name,
    version=model_version,
    description=f"The model version {model_version} was transitioned to {new_stage} on {date}"
)

<ModelVersion: aliases=['Beta'], creation_timestamp=1718878153758, current_stage='None', description='The model version 4 was transitioned to Beta on 2024-06-20', last_updated_timestamp=1718881765430, name='nyc_taxi_model', run_id='18f755f5614c4fd2be976fa7bfb9a0e1', run_link=None, source='/workspaces/mlops-zoomcamp/02-experiment-tracking/mlruns/1/18f755f5614c4fd2be976fa7bfb9a0e1/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=4>

### Comparing versions and selecting the new "Production" model

In the last section, we will retrieve models registered in the model registry and compare their performance on an unseen test set. The idea is to simulate the scenario in which a deployment engineer has to interact with the model registry to decide whether to update the model version that is in production or not.

These are the steps:

1. Load the test dataset, which corresponds to the NYC Green Taxi data from the month of March 2021.
2. Download the `DictVectorizer` that was fitted using the training data and saved to MLflow as an artifact, and load it with pickle.
3. Preprocess the test set using the `DictVectorizer` so we can properly feed the regressors.
4. Make predictions on the test set using the model versions that are currently in the "Staging" and "Production" stages, and compare their performance.
5. Based on the results, update the "Production" model version accordingly.


**Note: the model registry doesn't actually deploy the model to production when you transition a model to the "Production" stage, it just assign a label to that model version. You should complement the registry with some CI/CD code that does the actual deployment.**

In [50]:
from sklearn.metrics import mean_squared_error
import pandas as pd


def read_dataframe(filename):
    if filename.endswith('.csv'):
        df = pd.read_csv(filename)
        df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
        df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)
    elif filename.endswith('.parquet'):
        df = pd.read_parquet(filename)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df

def preprocess(df, dv):
    df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']
    categorical = ['PU_DO']
    numerical = ['trip_distance']
    train_dicts = df[categorical + numerical].to_dict(orient='records')
    return dv.transform(train_dicts)


def test_model(name, alias, X_test, y_test):
    model = mlflow.pyfunc.load_model(f"models:/{name}@{alias}")
    y_pred = model.predict(X_test)
    return {"rmse": mean_squared_error(y_test, y_pred, squared=False)}

In [31]:
df = read_dataframe('https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2024-03.parquet')

In [32]:
client.download_artifacts(run_id=run_id, path='preprocessor', dst_path='.')

'/workspaces/mlops-zoomcamp/02-experiment-tracking/preprocessor'

In [33]:
import pickle

with open("preprocessor/preprocessor.b", "rb") as f_in:
    dv = pickle.load(f_in)

In [34]:
X_test = preprocess(df, dv)

In [35]:
target = "duration"
y_test = df[target].values

In [51]:
%time test_model(name=model_name, alias="beta", X_test=X_test, y_test=y_test)

/opt/conda/envs/exp-tracking-env/lib/python3.9/site-packages/xgboost/core.py:160: UserWarning: [11:34:11] WARNING: /workspace/src/objective/regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)


CPU times: user 6.94 s, sys: 18.6 ms, total: 6.96 s
Wall time: 3.82 s


/opt/conda/envs/exp-tracking-env/lib/python3.9/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


{'rmse': 5.187936735881068}

In [52]:
%time test_model(name=model_name, alias="Sigma", X_test=X_test, y_test=y_test)

/opt/conda/envs/exp-tracking-env/lib/python3.9/site-packages/xgboost/core.py:160: UserWarning: [11:34:23] WARNING: /workspace/src/objective/regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)


CPU times: user 6.9 s, sys: 11.1 ms, total: 6.91 s
Wall time: 3.87 s


/opt/conda/envs/exp-tracking-env/lib/python3.9/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


{'rmse': 5.187936735881068}

In [53]:
client.set_registered_model_alias(
    name=model_name,
    version=4,
    alias="Production")